In [29]:
%config IPCompleter.greedy=True


In [30]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_train

,Id,label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0


In [31]:
import numpy as np

print(np.__path__)

train0 = np.load('train/train/0.npy')
train0

['C:\\Users\\CCS\\AppData\\Roaming\\Python\\Python35\\site-packages\\numpy']


array([[nan, nan,  1., ..., nan, nan, nan],
       [nan, nan,  2., ..., 19., nan, nan],
       [nan, nan,  3., ..., 22., nan, nan],
       ...,
       [nan, nan, 52., ..., 26., nan, nan],
       [nan, nan, 53., ..., 22., nan, nan],
       [nan, nan, 54., ..., 18., nan, nan]])

In [32]:
def load_dataframe(id):
    train_data = np.load("train/train/{}.npy".format(id))
    return pd.DataFrame(data=train_data)

In [33]:
def get_missing_data(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [34]:
def clear_missing_data(df, missing_indices):
    df = df.drop(missing_indices,1)
    # for col in df_temp.columns:
    #    df = df.drop(df.loc[df[col].isnull()].index)
    count = df.isnull().sum().max() #just checking that there's no missing data missing...
    if count > 0:
        print(count, "NaN")
        return pd.DataFrame()
    return df

In [35]:
def impute_missing_data_with_average(df, drop_indices, average):
    df = df.drop(drop_indices, 1)
    for col in df.columns:
        df[col] = df[col].fillna(average[col])
    return df

In [36]:
def impute_missing_data_with_zero(df, drop_indices):
    df = df.drop(drop_indices, 1)
    for col in df.columns:
        df[col] = df[col].fillna(0)
    return df

In [37]:
df8 = load_dataframe(8)
missing_df8 = get_missing_data(df8)
missing_indices = missing_df8[missing_df8['Total'] > 0].index.append(pd.Index([2,11,33,35]))
cleaned_df8 = clear_missing_data(df8, missing_indices)

#custom to add some column back
adding_cols = [22]
to_be_removed = []
for idx, i in enumerate(missing_indices):
    if i in adding_cols:
        to_be_removed.append(idx)
missing_indices = missing_indices.delete(to_be_removed)

missing_indices.sort_values(ascending=True)

Int64Index([ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 18,
            19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
            37, 38, 39],
           dtype='int64')

In [38]:
y = df_train['label']
y

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        1
9        0
10       1
11       0
12       0
13       0
14       1
15       0
16       0
17       1
18       0
19       0
20       0
21       1
22       0
23       0
24       0
25       0
26       0
27       1
28       0
29       0
        ..
30306    0
30307    0
30308    0
30309    0
30310    0
30311    0
30312    0
30313    0
30314    0
30315    0
30316    0
30317    0
30318    0
30319    0
30320    0
30321    1
30322    0
30323    0
30324    0
30325    0
30326    0
30327    0
30328    0
30329    0
30330    0
30331    0
30332    0
30333    1
30334    0
30335    1
Name: label, Length: 30336, dtype: int64

In [39]:
import sys

average = np.empty(40)
accumulated_count = [0] * df_train.shape[0]
whole_col_same_value = [35]
for id in df_train['Id']:
    if id % 10 == 0:
        sys.stdout.write("\rprocessing %i" % id)
#         sys.stdout.write("\r {} {}".format(id, average[1]))
        sys.stdout.flush()
    df = load_dataframe(id)
    for i in range(40):
        if i in whole_col_same_value:
            value = df.values[0][i]
            if np.isnan(value):
                continue
            average[i] = (average[i] * accumulated_count[i] + df.values[0][i]) / (accumulated_count[i] + 1)
            accumulated_count[i] += 1
        else:
            values = df.values[:,i]
            is_not_nan = ~np.isnan(values)
            values = values[is_not_nan]
            if len(values) == 0:
                continue
            count = len(values)
            average[i] = (average[i] * accumulated_count[i] + np.sum(values)) / (accumulated_count[i] + count)
            accumulated_count[i] += count
    
average
    

processing 30330

array([  0.49555807,   2.04680609,  27.15729269, 122.83364621,
         2.28476449,   2.57970877,  10.49785743,   0.50444193,
         4.14589169,   1.46722637,   2.13536761,  62.34371757,
       135.96647116,   0.55852966,   7.79536143,  84.76420289,
       289.8111412 ,  81.18010694,   7.37931228,   3.561608  ,
        92.04059017,  24.04156655,  41.11530986, 105.79398754,
        62.82663102,  33.57663965,  -0.66819929, 197.20136227,
       106.01175318,  97.22616487, 291.42421967,  30.76740307,
        11.60964474,   0.56573611,   8.7685371 , -50.54944157,
        24.08684309,  18.74176613,  41.006491  ,  36.99253031])

In [40]:
dataframes = []
empty_dfi = []

for id in df_train['Id']:
    if id % 10 == 0:
        sys.stdout.write("\rprocessing %i" % id)
    df = load_dataframe(id)
    
    #remove any columns with nan
#     dfi = clear_missing_data(df, missing_indices)
#     if dfi.empty:
#         empty_dfi.append(id)
#         avg_data = {}http://localhost:8888/notebooks/Desktop/cs5228project/data_exploration.ipynb#
#         for missing_id in np.delete(np.array(range(40)), missing_indices):
#             avg_data[missing_id] = np.full(df.shape[0],average[missing_id])
#         dfi = pd.DataFrame(data=avg_data)
    
    #remove only if given and impute some values
    dfi = impute_missing_data_with_average(df, missing_indices, average)
    
    #remove only if given and set NaN as 0
#     dfi = impute_missing_data_with_zero(df, missing_indices)
    
    dfi['Id'] = id
    
    dataframes.append(dfi)
print("\nempty dfi", empty_dfi)
df_train_all = pd.concat(dataframes, ignore_index=True)

processing 30330
empty dfi []


In [41]:
df_train_all

,5,13,22,Id
0,2.579709,0.55853,41.11531,0
1,2.579709,0.55853,41.11531,0
2,2.579709,0.55853,41.11531,0
3,2.579709,0.55853,100.00000,0
4,2.579709,0.28000,41.11531,0
5,2.579709,0.55853,41.11531,0
6,2.579709,0.55853,41.11531,0
7,2.579709,0.55853,41.11531,0
8,2.579709,0.55853,41.11531,0
9,2.579709,0.55853,41.11531,0


In [42]:
from tsfresh import extract_features
 
extracted_features = extract_features(df_train_all, column_id="Id", show_warnings=False)
extracted_features

Feature Extraction: 100%|███████████████████████████████████████████████████████████| 20/20 [1:10:03<00:00, 210.20s/it]


MemoryError: Unable to allocate array with shape (72260352,) and data type int64

In [ ]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

# print(extracted_features.shape, y.shape)
impute(extracted_features)
features_filtered = select_features(extracted_features, y)

In [ ]:
from sklearn import preprocessing

XTrain = preprocessing.scale(features_filtered)
XTrain

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

gb_clf = GradientBoostingClassifier()
scores = cross_val_score(gb_clf, XTrain, y, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
scores

In [ ]:
gb_clf.fit(XTrain, y)

In [ ]:
def load_dataframe_test(id):
    train_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=train_data)

df_test = pd.read_csv('sample_solution.csv')
dataframestest = []
empty_dfi = []
for id in df_test['Id']:
    if id % 10 == 0:
        sys.stdout.write("\rprocessing %i" % id)
    df = load_dataframe_test(id)
    
    #remove any columns with nan
#     dfi = clear_missing_data(df, missing_indices)
#     if dfi.empty:
# #         print("{}:".format(id))
#         empty_dfi.append(id)
#         continue
    #remove only if given and impute some values
    dfi = impute_missing_data_with_average(df, missing_indices, average)

    #remove only if given and set NaN as 0
#     dfi = impute_missing_data_with_zero(df, missing_indices)
    
    dfi['Id'] = id
    dataframestest.append(dfi)

print("\nempty dfi", empty_dfi)
df_test_all = pd.concat(dataframestest, ignore_index=True)

In [ ]:
df_test_all

In [ ]:
extracted_features_test = extract_features(df_test_all, column_id="Id")
extracted_features_test

In [ ]:
XTest = extracted_features_test[features_filtered.columns]
XTest

In [ ]:
XTest = extracted_features_test[features_filtered.columns]
XTest = XTest.replace([np.inf], np.finfo('float32').max).replace([np.inf, -np.inf], np.finfo('float32').min).fillna(0)
print(np.all(np.isfinite(XTest)))
# XTest

In [ ]:
XTest = preprocessing.scale(XTest)
XTest

In [ ]:
probs = gb_clf.predict_proba(XTest)
probs

In [ ]:
res = probs[:, 0]
res

In [ ]:
df_test['Predicted'] = res
df_test

In [ ]:
df_test.to_csv('test.csv', index=False)